<font size="4">Reinforcement learning - <b>Naughts and Crosses</b></font>

Welcome! Here is my attempt to use Reinforcement learning for <b>naughts and crosses</b> problem. Of course this problem could be decided recursively using <b>Minimax</b> but this game is taken for understanding for applying the same ideas for chess or checkers.

In [9]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

In [11]:
tf.__version__

'2.9.0'

First of all I will introduce Minimax algorithm just to make a strong opponent for our neural network.

In [2]:
# Minimax function algorithm
# position is a vector of board, where 
# - 0 - empty square
# - 1 - X
# - -1 - O

def calc_position(position):
    # condition of winning - 3 in a row (8 possible cases)
    if (position[0] == position[1] == position[2] != 0) or (position[3] == position[4] == position[5] != 0) or \
        (position[6] == position[7] == position[8] != 0) or (position[0] == position[3] == position[6] != 0) or \
        (position[1] == position[4] == position[7] != 0) or (position[2] == position[5] == position[8] != 0) or \
        (position[0] == position[4] == position[8] != 0) or (position[2] == position[4] == position[6] != 0):
        all_positions[tuple(position)]=[[],[],sum(position)*2-1]
        return None, sum(position)*2-1 # return -1 if O wins and 1 if X wins
    
    if 0 not in position:
        all_positions[tuple(position)]=[[],[],0]
        return None, 0 # return 0 as draw
    
    # turn to move
    turn=1
    if sum(position)!=0: # if sum of positions not equals 0 - then move of O
        turn=-1
    
    # save moves and values of the positions afrer each move
    moves=[]
    values=[]
    for i in range(9):
        if position[i]==0:
            sub_position=position[:]
            sub_position[i]=turn
            moves.append(i)
            _,value = calc_position(sub_position)
            values.append(value)
            
    if turn==1: 
        best_value=max(values)
        index=np.argmax(values)
    else:
        best_value=min(values)
        index=np.argmin(values)
        
    good_moves=[] # Save bad and good moves to lists
    bad_moves=[]
    for i,value in enumerate (values):
        if value==best_value:
            good_moves.append(moves[i])
        else:
            bad_moves.append(moves[i])
    all_positions[tuple(position)]=[good_moves,bad_moves,values[index]]
    return good_moves,values[index]

In [3]:
# Will try to play from the beginning position
all_positions = {}

position = [0,0,0,
            0,0,0,
            0,0,0]

calc_position(position)

([0, 1, 2, 3, 4, 5, 6, 7, 8], 0)

As we see if <b>X</b> and <b>O</b> will play best way the result will be draw for every of the first possible moves

In [4]:
# After the first move of X in the center
position = [0,0,0,
            0,1,0,
            0,0,0]

calc_position(position)

([0, 2, 6, 8], 0)

In the position below <b>O</b> has only 4 moves of 8 to make a draw

Now we are ready to start teaching! But first it's better to save all possible positions with bag and good moves to dictionary because if will take few seconds to compute the best move for each game.

For this case we can use the same <b>calc_position()</b> function just uncomment all lines with <b>all_positions[tuple(position)]=...</b>.

In [5]:
all_positions={}
calc_position([0,0,0,0,0,0,0,0,0])

([0, 1, 2, 3, 4, 5, 6, 7, 8], 0)

In [6]:
all_positions[(0,-1,0,0,1,0,0,0,0)]

[[0, 2, 3, 5, 6, 8], [7], 1]

The dictionary <b>all_positions</b> contains tuples of all possible positions as keys and as values the list of best moves, the list of other moves and a result after the best play for both sides.

In [7]:
# Model as Tensorflow graph

# Weights and biases of multilayer perceptron
W1 = tf.Variable(tf.random.truncated_normal([9, 30], stddev=0.1))
W2 = tf.Variable(tf.random.truncated_normal([30, 10], stddev=0.1))
W3 = tf.Variable(tf.random.truncated_normal([10, 1], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, shape=[30]))
B2 = tf.Variable(tf.constant(0.1, shape=[10]))
B3 = tf.Variable(tf.constant(0.1, shape=[1]))

#x  = tf.placeholder(tf.float32, [None, 9], name='x') # input
#y_ = tf.placeholder(tf.float32, [None, 1],  name='y_') # label

#g  = tf.placeholder(tf.float32, [None], name='gamma') # reward function coefficients

# model prediction for current move
def forward(x,W1,W2,W3,B1,B2,B3,y_,g):
    res = tf.nn.tanh(tf.matmul(x,W1)+B1)
    res = tf.nn.tanh(tf.matmul(res,W2)+B2)
    res = tf.nn.tanh(tf.matmul(res,W3)+B3)
    # reward function for a set of moves
    y = tf.reduce_sum(res*g)
    # mse loss function
    loss = (y_-y)**2
    return loss,res

Teaching the neural network!

In [8]:
# gamma coefficients of the reward function
k=0.8
gamma=np.zeros(5)
for i in range(len(gamma)):
    gamma[i]=np.power(k,i+1)
mistake_rate = 0.1 # It will be good to let the compluter to lose sometimes
alpha = 0.001
epochs = 10000

for games in tqdm(range(epochs)):
    # Now computer plays for O
    
    current_position=list(np.zeros([9])) # Starting position is always the same
    x=[] # input matrix
    y=np.zeros([1,1]) # label
    
    while True:
        good_moves,bad_moves,result=all_positions[tuple(current_position)]
        
        if len(good_moves)+len(bad_moves)==0: # if no moves then the game is finished
            y = result
            break
            
        positions=[] # we need to predict the value of possible moves
        for move in good_moves:
            positions.append(current_position[:])
            positions[-1][move]=1
        for move in bad_moves:
            positions.append(current_position[:])
            positions[-1][move]=1
          
        positions = np.array(positions, dtype=np.float32)
        loss, r = forward(positions,W1,W2,W3,B1,B2,B3,y,gamma[:len(positions)])
        r = np.array(r)
        r = (r+1)/sum(r+1) # Normalizing predictions
        
        move = np.random.choice(good_moves+bad_moves,p=r[:,0]) # Choose the move by random choice
        current_position[move] = 1
        x.append(current_position[:])
        
        # Now we generate the reply of the computer
            
        good_moves,bad_moves,result=all_positions[tuple(current_position)]
        
        if len(good_moves)+len(bad_moves)==0:
            y = result
            break
        
        if len(bad_moves)>0 and np.random.rand()<=mistake_rate: # According to the mistake_rate make a mistake
            move = np.random.choice(bad_moves)
        else:
            move = np.random.choice(good_moves)
        
        current_position[move] = -1
    
    # Train
    x = np.array(x, dtype=np.float32)

    for j in range(10):
        with tf.GradientTape() as g:
            g.watch([W1,W2,W3,B1,B2,B3])
            loss, r = forward(x,W1,W2,W3,B1,B2,B3,y,gamma[0:len(x)])
            #print(loss)

        dJ = g.gradient(loss, [W1,W2,W3,B1,B2,B3])
        W1 = W1 - alpha*dJ[0]
        W2 = W2 - alpha*dJ[1]
        W3 = W3 - alpha*dJ[2]
        B1 = B1 - alpha*dJ[3]
        B2 = B2 - alpha*dJ[4]
        B3 = B3 - alpha*dJ[5]

# Now computer plays for X. The same code but without predictions

    current_position=list(np.zeros([9]))
    x=[]
    y=np.zeros([1,1])

    while True:
        good_moves,bad_moves,result=all_positions[tuple(current_position)]
        
        if len(good_moves)+len(bad_moves)==0:
            y = result
            break
        
        if len(bad_moves)>0 and np.random.rand()<=mistake_rate:
            move = np.random.choice(bad_moves)
        else:
            move = np.random.choice(good_moves)
        
        current_position[move] = 1
        x.append(current_position[:])
        
        good_moves,bad_moves,result=all_positions[tuple(current_position)]
        
        if len(good_moves)+len(bad_moves)==0:
            y = result
            break
        
        if len(bad_moves)>0 and np.random.rand()<=mistake_rate:
            move = np.random.choice(bad_moves)
        else:
            move = np.random.choice(good_moves)
        
        current_position[move] = -1
        
    x = np.array(x, dtype=np.float32)
    
    for j in range(10):
        with tf.GradientTape() as g:
            g.watch([W1,W2,W3,B1,B2,B3])
            loss, r = forward(x,W1,W2,W3,B1,B2,B3,y,gamma[0:len(x)])
            #print(loss)

        dJ = g.gradient(loss, [W1,W2,W3,B1,B2,B3])
        W1 = W1 - alpha*dJ[0]
        W2 = W2 - alpha*dJ[1]
        W3 = W3 - alpha*dJ[2]
        B1 = B1 - alpha*dJ[3]
        B2 = B2 - alpha*dJ[4]
        B3 = B3 - alpha*dJ[5]

100%|██████████| 10000/10000 [06:21<00:00, 26.22it/s]


Now we are ready to test our model. For that we'll generate 1000 games and will see what will happen

In [10]:
# 1000 games with best play of black

draws=0
loses=0
wins=0
mistake_rate=0.1 # mistake rate for the computer
had_win=0 # count how many winning positions the neural network had
had_lose=0 # count how many lost positions the neural network had

for i in tqdm(range(1000)):
    current_position=list(np.zeros([9]))
    had_win_t=0
    had_lose_t=0
    
    while True:
        good_moves,bad_moves,result=all_positions[tuple(current_position)]
        
        if result==-1:
            had_lose_t=1
        if result==1:
            had_win_t=1
            
        if len(good_moves)+len(bad_moves)==0:
            if result==0:
                draws+=1
            if result==-1:
                loses+=1
            break
            
        positions=[]
        for move in good_moves:
            positions.append(current_position[:])
            positions[-1][move]=1
        for move in bad_moves:
            positions.append(current_position[:])
            positions[-1][move]=1
            
        positions = np.array(positions, dtype=np.float32)
        loss, r = forward(positions,W1,W2,W3,B1,B2,B3,y,gamma[:len(positions)])
        r = np.array(r)
        r = (r+1)/sum(r+1) # Normalizing predictions
        
        move=(good_moves+bad_moves)[np.argmax(r)]
        current_position[move]=1
             
        good_moves,bad_moves,result=all_positions[tuple(current_position)]
        
        if result==-1:
            had_lose_t=1
        if result==1:
            had_win_t=1
            
        if len(good_moves)+len(bad_moves)==0:
            if result==0:
                draws+=1
            if result==1:
                wins+=1
            break
        
        if len(bad_moves)>0 and np.random.rand()<mistake_rate:
            move=np.random.choice(bad_moves)
        else:
            move=np.random.choice(good_moves)
        
        current_position[move]=-1
            
    had_win+=had_win_t
    had_lose+=had_lose_t  

100%|██████████| 1000/1000 [00:01<00:00, 699.99it/s]


In [11]:
print("Games won:", wins)
print("Games drawn:", draws)
print("Games lost:", loses)
print("Had winning position:", had_win)
print("Had lost position:", had_lose)

Games won: 244
Games drawn: 748
Games lost: 8
Had winning position: 315
Had lost position: 10
